In [1]:
# Usage: $env:PYTHONPATH="."; python Classical_Models\Lasso\lasso_train.py

import os
import time
import pandas as pd
import numpy as np
from sklearn.linear_model import MultiTaskLasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import joblib

# === CONFIG ===
FAST_MODE = True
TRAIN_FILE = r"C:\Users\pc\Desktop\PROJECT_THESIS_Thrisha_Rajkumar\data\processed\train_for_model.csv"
HOLDOUT_FILE = r"C:\Users\pc\Desktop\PROJECT_THESIS_Thrisha_Rajkumar\data\processed\merged_test_with_features.csv"
MODEL_DIR = r"C:\Users\pc\Desktop\PROJECT_THESIS_Thrisha_Rajkumar\Classical_Models\models\lasso\testing"
os.makedirs(MODEL_DIR, exist_ok=True)

TARGET_COLUMNS = [
    'voltage_rise_time_pulse1', 'voltage_rise_time_pulse2',
    'voltage_fall_time_pulse1', 'voltage_fall_time_pulse2',
    'current_rise_time_pulse1', 'current_rise_time_pulse2',
    'current_fall_time_pulse1', 'current_fall_time_pulse2',
    'overshoot_pulse_1', 'overshoot_pulse_2',
    'undershoot_pulse_1', 'undershoot_pulse_2',
    'ringing_frequency_MHz'
]
DROP_COLUMNS = ['DeviceID']

# === 1. Load & Sample ===
df = pd.read_csv(TRAIN_FILE)
df.dropna(subset=TARGET_COLUMNS, inplace=True)
if 'Device' in df.columns:
    df = df.drop(columns=['Device'])

if FAST_MODE:
    df = df.sample(frac=0.25, random_state=42)

X = df.drop(columns=DROP_COLUMNS + TARGET_COLUMNS)
y = df[TARGET_COLUMNS]
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=42)

# === 2. Scaling ===
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_val_scaled = scaler_X.transform(X_val)
X_test_scaled = scaler_X.transform(X_test)
y_train_scaled = scaler_y.fit_transform(y_train)
y_val_scaled = scaler_y.transform(y_val)
y_test_scaled = scaler_y.transform(y_test)

joblib.dump(scaler_X, os.path.join(MODEL_DIR, "input_scaler.pkl"))
joblib.dump(scaler_y, os.path.join(MODEL_DIR, "output_scaler.pkl"))

# === 3. Train Lasso ===
model = MultiTaskLasso(alpha=0.001, max_iter=1000)

start_time = time.time()
model.fit(X_train_scaled, y_train_scaled)
print(f"Training completed in {time.time() - start_time:.2f} seconds")

# === 4. Evaluation ===
def evaluate(model, X_scaled, y_scaled, label, save_name):
    y_pred_scaled = model.predict(X_scaled)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_scaled)

    rmse = np.sqrt(mean_squared_error(y_true, y_pred, multioutput='raw_values'))
    r2 = r2_score(y_true, y_pred, multioutput='raw_values')

    print(f"\n{label} Evaluation:")
    print(f"{'Target Output':<35} {'RMSE':>15} {'R² Score':>12}")
    print("-" * 64)
    for i, col in enumerate(TARGET_COLUMNS):
        print(f"{col:<35} {rmse[i]:.2E} {r2[i]:>12.4f}")

    # Save predictions
    df_result = pd.DataFrame({f"{col}_actual": y_true[:, i] for i, col in enumerate(TARGET_COLUMNS)})
    for i, col in enumerate(TARGET_COLUMNS):
        df_result[f"{col}_predicted"] = y_pred[:, i]
    df_result.to_csv(os.path.join(MODEL_DIR, save_name), index=False)

    # Save metrics
    df_metrics = pd.DataFrame({
        "Target": TARGET_COLUMNS,
        "RMSE": [f"{val:.2E}" for val in rmse],
        "R2_Score": [round(val, 4) for val in r2]
    })
    metrics_path = os.path.join(MODEL_DIR, save_name.replace(".csv", "_metrics.csv"))
    df_metrics.to_csv(metrics_path, index=False)
    print(f"Saved predictions to: {save_name}")
    print(f"Saved metrics to: {metrics_path}")

# === 5. Evaluate All Splits ===
evaluate(model, X_val_scaled, y_val_scaled, "Validation", "validation_predictions.csv")
evaluate(model, X_test_scaled, y_test_scaled, "Internal Test", "internal_test_predictions.csv")

# === 6. Holdout Test ===
df_holdout = pd.read_csv(HOLDOUT_FILE)
df_holdout.dropna(subset=TARGET_COLUMNS, inplace=True)
if 'Device' in df_holdout.columns:
    df_holdout = df_holdout.drop(columns=['Device'])

X_holdout = df_holdout.drop(columns=DROP_COLUMNS + TARGET_COLUMNS)
y_holdout = df_holdout[TARGET_COLUMNS]
X_holdout_scaled = scaler_X.transform(X_holdout)
y_holdout_scaled = scaler_y.transform(y_holdout)
evaluate(model, X_holdout_scaled, y_holdout_scaled, "External Holdout", "holdout_predictions.csv")

# === 7. Save Model ===
joblib.dump(model, os.path.join(MODEL_DIR, "model.pkl"))
print(f"\nModel and scalers saved to: {MODEL_DIR}")


c:\Users\pc\anaconda3\envs\text_analytics\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:2637: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 41892.798571304505, tolerance: 109.76940000000026
  ) = cd_fast.enet_coordinate_descent_multi_task(


Training completed in 178.56 seconds

Validation Evaluation:
Target Output                                  RMSE     R² Score
----------------------------------------------------------------
voltage_rise_time_pulse1            3.74E-08       0.0131
voltage_rise_time_pulse2            3.28E-06       0.9436
voltage_fall_time_pulse1            5.48E-07       0.9922
voltage_fall_time_pulse2            6.46E-07       0.9912
current_rise_time_pulse1            3.09E-06       0.9241
current_rise_time_pulse2            3.55E-07       0.6998
current_fall_time_pulse1            9.08E-06       0.3678
current_fall_time_pulse2            3.68E-08       0.1853
overshoot_pulse_1                   5.90E+00       0.9515
overshoot_pulse_2                   4.72E+01       0.9769
undershoot_pulse_1                  1.35E+01       0.7297
undershoot_pulse_2                  2.63E+01       0.5770
ringing_frequency_MHz               3.28E+00       0.9946
Saved predictions to: validation_predictions.csv
Saved 

In [2]:
# Usage: $env:PYTHONPATH="."; python Classical_Models\Lasso\lasso_generalization.py

import os
import time
import pandas as pd
import numpy as np
from sklearn.linear_model import MultiTaskLasso
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import joblib

# === CONFIG ===
FAST_MODE = True
TRAIN_FILE = r"C:\Users\pc\Desktop\PROJECT_THESIS_Thrisha_Rajkumar\data\raw\merged_train_5_MOSFETS.csv"
TEST_FILE = r"C:\Users\pc\Desktop\PROJECT_THESIS_Thrisha_Rajkumar\data\raw\merged_test_1_MOSFET.csv"
MODEL_DIR = r"C:\Users\pc\Desktop\PROJECT_THESIS_Thrisha_Rajkumar\Classical_Models\models\lasso\generalization"
os.makedirs(MODEL_DIR, exist_ok=True)

TARGET_COLUMNS = [
    'voltage_rise_time_pulse1', 'voltage_rise_time_pulse2',
    'voltage_fall_time_pulse1', 'voltage_fall_time_pulse2',
    'current_rise_time_pulse1', 'current_rise_time_pulse2',
    'current_fall_time_pulse1', 'current_fall_time_pulse2',
    'overshoot_pulse_1', 'overshoot_pulse_2',
    'undershoot_pulse_1', 'undershoot_pulse_2',
    'ringing_frequency_MHz'
]
DROP_COLUMNS = ['DeviceID']

# === 1. Load & Sample Train Data ===
df_train = pd.read_csv(TRAIN_FILE)
df_train.dropna(subset=TARGET_COLUMNS, inplace=True)
if 'Device' in df_train.columns:
    df_train = df_train.drop(columns=['Device'])

if FAST_MODE:
    df_train = df_train.sample(frac=0.25, random_state=42)

X_train = df_train.drop(columns=DROP_COLUMNS + TARGET_COLUMNS)
y_train = df_train[TARGET_COLUMNS]

# === 2. Load Test Data (Unseen MOSFET) ===
df_test = pd.read_csv(TEST_FILE)
df_test.dropna(subset=TARGET_COLUMNS, inplace=True)
if 'Device' in df_test.columns:
    df_test = df_test.drop(columns=['Device'])

X_test = df_test.drop(columns=DROP_COLUMNS + TARGET_COLUMNS)
y_test = df_test[TARGET_COLUMNS]

# === 3. Scaling ===
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)
y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)

joblib.dump(scaler_X, os.path.join(MODEL_DIR, "input_scaler.pkl"))
joblib.dump(scaler_y, os.path.join(MODEL_DIR, "output_scaler.pkl"))

# === 4. Train Lasso ===
model = MultiTaskLasso(alpha=0.001, max_iter=1000)
start_time = time.time()
model.fit(X_train_scaled, y_train_scaled)
print(f"Training completed in {time.time() - start_time:.2f} seconds")

# === 5. Evaluate ===
def evaluate(model, X_scaled, y_scaled, label, save_name):
    y_pred_scaled = model.predict(X_scaled)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_scaled)

    rmse = np.sqrt(mean_squared_error(y_true, y_pred, multioutput='raw_values'))
    r2 = r2_score(y_true, y_pred, multioutput='raw_values')

    print(f"\n{label} Evaluation:")
    print(f"{'Target Output':<35} {'RMSE':>15} {'R² Score':>12}")
    print("-" * 64)
    for i, col in enumerate(TARGET_COLUMNS):
        print(f"{col:<35} {rmse[i]:.2E} {r2[i]:>12.4f}")

    # Save predictions
    df_result = pd.DataFrame({f"{col}_actual": y_true[:, i] for i, col in enumerate(TARGET_COLUMNS)})
    for i, col in enumerate(TARGET_COLUMNS):
        df_result[f"{col}_predicted"] = y_pred[:, i]
    df_result.to_csv(os.path.join(MODEL_DIR, save_name), index=False)

    # Save metrics
    df_metrics = pd.DataFrame({
        "Target": TARGET_COLUMNS,
        "RMSE": [f"{val:.2E}" for val in rmse],
        "R2_Score": [round(val, 4) for val in r2]
    })
    metrics_path = os.path.join(MODEL_DIR, save_name.replace(".csv", "_metrics.csv"))
    df_metrics.to_csv(metrics_path, index=False)
    print(f"Saved predictions to: {save_name}")
    print(f"Saved metrics to: {metrics_path}")

# === 6. Evaluate Generalization ===
evaluate(model, X_test_scaled, y_test_scaled, "Generalization (Unseen MOSFET)", "unseen_predictions.csv")

# === 7. Save Model ===
joblib.dump(model, os.path.join(MODEL_DIR, "model.pkl"))
print(f"\nModel and scalers saved to: {MODEL_DIR}")


c:\Users\pc\anaconda3\envs\text_analytics\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:2637: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 74157.78855686384, tolerance: 156.53430000000054
  ) = cd_fast.enet_coordinate_descent_multi_task(


Training completed in 299.06 seconds

Generalization (Unseen MOSFET) Evaluation:
Target Output                                  RMSE     R² Score
----------------------------------------------------------------
voltage_rise_time_pulse1            2.29E-08      -0.1302
voltage_rise_time_pulse2            4.08E-06    -107.7216
voltage_fall_time_pulse1            1.66E-06 -263018.8946
voltage_fall_time_pulse2            1.85E-06    -329.4717
current_rise_time_pulse1            4.67E-06      -9.9458
current_rise_time_pulse2            1.49E-06      -2.3770
current_fall_time_pulse1            1.12E-05      -0.4063
current_fall_time_pulse2            2.91E-08      -0.2539
overshoot_pulse_1                   1.01E+01       0.8399
overshoot_pulse_2                   4.70E+01      -0.2952
undershoot_pulse_1                  6.86E+02    -574.0507
undershoot_pulse_2                  3.81E+01       0.2129
ringing_frequency_MHz               5.37E+01     -63.8709
Saved predictions to: unseen_predic